In [103]:
import jax.numpy as jnp
import numpy as np
from jax import grad, hessian
from jax import jit

In [104]:
GlobalSeed = 4
ErrorsLevel = 10
FunctionComplexity = 5
Degree = 10

In [105]:
np.random.seed(GlobalSeed)
def predict(params,x):
    if(Degree == 10):
        return params[0] + params[1] * x + params[2] * x ** 2 + params[3] * x ** 3 + params[4] * x ** 4 + params[5] * x ** 5 + params[6] * x ** 6 + params[7] * x ** 7 + params[8] * x ** 8 + params[9] * x ** 9 + params[10] * x ** 10
    elif(Degree == 3):
        return params[0] + params[1] * x + params[2] * x ** 2 + params[3] * x ** 3
def generate_points(params,n, error):
    X = np.random.rand(n)
    Y = predict(params,X) + error*np.random.randn(n)
    return X, Y
params = np.random.rand(Degree + 1)*FunctionComplexity
X, Y = generate_points(params,100, 1.5)
X_org = jnp.linspace(min(X), max(X), 100)
Y_org = predict(params,X_org)
def loss(params, X, Y):
    predictions = predict(params, X)
    return jnp.mean((predictions - Y) ** 2)
grad_loss = jit(grad(loss))
hessian_loss = hessian(loss)
def newton_method(X, Y, num_iterations=10):
    params = np.zeros(Degree + 1)
    for i in range(num_iterations):
        grad_l = grad_loss(params, X, Y)
        hess_l = hessian_loss(params, X, Y)
        params_new = params - jnp.linalg.solve(hess_l, grad_l)
        params = params_new
    return params
def showNewton(X, Y):
    params = newton_method(X, Y)
    error = Y - predict(params, X)
    X_fit = jnp.linspace(min(X), max(X), 100)
    Y_fit = predict(params, X_fit)
    return X_fit,Y_fit,error
def AddOutliers(X, Y):
    np.random.seed(GlobalSeed)
    std_dev = np.std(Y)
    X_outliers = np.random.rand(5)
    Y_outliers = predict(params, X_outliers) + np.random.uniform(-ErrorsLevel * std_dev, ErrorsLevel * std_dev, len(X_outliers))
    X_ol = np.concatenate((X, X_outliers))
    Y_ol = np.concatenate((Y, Y_outliers))
    return X_ol, Y_ol
def GaussianCalc(X):
    ave = jnp.mean(X)
    std = jnp.std(X)
    return 1/(std*jnp.sqrt(2*jnp.pi))*jnp.exp(-0.5*((X-ave)/std)**2)
def removeOutliers(X, Y, error,threshold=2):
    ave = jnp.mean(error)
    std = jnp.std(error)
    X_new = []
    Y_new = []
    counter = 0
    for i in range(len(error)):
        if error[i] < threshold*std and error[i] > -threshold*std:
            counter+=1
            X_new.append(X[i])
            Y_new.append(Y[i])
    return jnp.array(X_new), jnp.array(Y_new)
def weighted_loss(params, X, Y, error):
    weighted_error = GaussianCalc(error)
    predictions = predict(params, X)
    return jnp.mean((predictions - Y) ** 2 * weighted_error)
grad_weighted_loss = jit(grad(weighted_loss))
hessian_weighted_loss = hessian(weighted_loss)
def newton_method_weighted(X, Y, num_iterations=10, tol=1e-6):
    params = np.zeros(Degree + 1)
    for i in range(num_iterations):
        error = Y - predict(params,X)
        grad_weighted_l = grad_weighted_loss(params, X, Y,error)
        hess_weighted_l = hessian_weighted_loss(params, X, Y,error)
        params_new = params - jnp.linalg.solve(hess_weighted_l, grad_weighted_l)
        params = params_new
    return params
def showNewton_weighted(X, Y,error):
    params = newton_method_weighted(X, Y, 10,error)
    X_fit = jnp.linspace(min(X), max(X), 100)
    Y_fit = predict(params, X_fit)
    return X_fit,Y_fit,error
from sklearn.metrics import mean_squared_error
def plot_and_calculate_rmse(plot_config1, plot_config2, title):
    rmse = np.sqrt(mean_squared_error(plot_config1.y, plot_config2.y))
    return rmse

In [106]:
X_ol = AddOutliers(X,Y)[0]
Y_ol = AddOutliers(X,Y)[1]
X_ol_fit,Y_ol_fit,error_ol_fit = showNewton(X_ol,Y_ol)
X_new2, Y_new2 = removeOutliers(X_ol, Y_ol, error_ol_fit,2)
X_ol_removed_fit2,Y_ol_removed_fit2,error_ol_removed_fit2 = showNewton(X_new2,Y_new2)
X_new1, Y_new1 = removeOutliers(X_ol, Y_ol, error_ol_fit,1)
X_ol_removed_fit1,Y_ol_removed_fit1,error_ol_removed_fit1 = showNewton(X_new1,Y_new1)
X_new3, Y_new3 = removeOutliers(X_ol, Y_ol, error_ol_fit,3)
X_ol_removed_fit3,Y_ol_removed_fit3,error_ol_removed_fit3 = showNewton(X_new3,Y_new3)
X_weighted_fit,Y_weighted_fit,error_ol_weighted_fit = showNewton_weighted(X_ol,Y_ol,error_ol_fit)
# Function to calculate RMSE
def calculate_rmse(y_true, y_pred):
    return jnp.mean((y_true - y_pred) ** 2)

# Generate a common set of X values for comparison
X_comp = jnp.linspace(min(X), max(X), 100)

# Compute RMSE for each fitted curve
rmse_ol = float(calculate_rmse(Y_org, Y_ol_fit))
rmse_ol_removed2 = float(calculate_rmse(Y_org, Y_ol_removed_fit2))
rmse_ol_removed1 = float(calculate_rmse(Y_org, Y_ol_removed_fit1))
rmse_ol_removed3 = float(calculate_rmse(Y_org, Y_ol_removed_fit3))
rmse_ol_weighted = float(calculate_rmse(Y_org, Y_weighted_fit))
import pandas as pd
data = {'Root Mean Square Error': [rmse_ol, rmse_ol_removed1, rmse_ol_removed2, rmse_ol_removed3, rmse_ol_weighted]}
df = pd.DataFrame(data, index=[
    'With Outliers',
    'Outliers Removed 1',
    'Outliers Removed 2',
    'Outliers Removed 3',
    'Weighted Loss'
])
df

,Root Mean Square Error
With Outliers,14.892470
Outliers Removed 1,1.946028
Outliers Removed 2,1585.532471
Outliers Removed 3,1.996939
Weighted Loss,4.594171
